<a href="https://colab.research.google.com/github/RomanGustavo/Mestrado----Projeto-Carro/blob/main/TWICE_Colab_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TWICE Dataset — Colab Starter
Este notebook prepara o ambiente, baixa o **subset** do TWICE, lê `.osi`, projeta cuboids (via utilitários do repo) e mostra como visualizar radar e rodar um detector (YOLOv7) para DEMO.

> Dica: execute célula por célula, na ordem. Se o runtime reiniciar, rode novamente as células de instalação (1–2).


In [ ]:
# CEL 1 — pacotes base
!pip -q install --upgrade opencv-python tqdm open3d protobuf==4.25.3 matplotlib

In [ ]:
# CEL 2 — Open Simulation Interface (OSI): tenta importar; se não existir, compila a partir do repo oficial
try:
    import osi3
    print("✅ osi3 já disponível")
except Exception as e:
    print("⚠️ osi3 não encontrado. Clonando OSI e instalando a partir do código-fonte...")
    !git clone -q https://github.com/OpenSimulationInterface/open-simulation-interface.git
    %cd /content/open-simulation-interface
    !python3 setup.py -q build_py
    !pip -q install .
    %cd /content
    import osi3
    print("✅ osi3 instalado")

In [ ]:
# CEL 3 — clonar scripts
!git clone -q https://github.com/TWICEdataset/scripts.git
%cd /content/scripts
!ls -la

In [ ]:
# CEL 4 — baixar o subset (~439 MB) e extrair
import os, zipfile, urllib.request, pathlib

subset_url = "https://twice.eletrica.ufpr.br/TWICEsubset.zip"
zip_path = "/content/TWICEsubset.zip"
out_dir = pathlib.Path("/content/TWICE")

if not os.path.exists(zip_path):
    print("⬇️ Baixando subset...")
    urllib.request.urlretrieve(subset_url, zip_path)
else:
    print("ZIP já existe. Pulando download.")

print("📦 Extraindo...")
out_dir.mkdir(parents=True, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(out_dir)

print("✅ Pronto. Dados em:", out_dir)

In [ ]:
# CEL 5 — paths e imports
import sys, pathlib
DATA_ROOT = pathlib.Path("/content/TWICE")
SCRIPTS_ROOT = pathlib.Path("/content/scripts")
sys.path.insert(0, str(SCRIPTS_ROOT))

print("DATA_ROOT =", DATA_ROOT)
print("SCRIPTS_ROOT =", SCRIPTS_ROOT)

from cuboid_project import rotation_matrix, cuboid_corners
from vehicle_to_image import Camera
print("✅ utilitários importados")

In [ ]:
# CEL 6 — iterar frames do .osi e salvar um MP4 (preview)
import cv2, struct
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Preferir as classes do repositório, quando disponíveis
try:
    from cuboid_project import SensorData as OSISensorData
    SensorData = OSISensorData
    from cuboid_project import SensorDataSeries  # se necessário em outros sensores
except Exception as e:
    SensorData = None
    print("Aviso: classes de Sensor do repo não importadas:", e)

def iter_camera_frames(osi_path: Path):
    assert SensorData is not None, "SensorData do repo não disponível — verifique imports na CEL 5."
    with open(osi_path, "rb") as f:
        while True:
            len_bytes = f.read(4)
            if not len_bytes:
                break
            (msg_len,) = struct.unpack("I", len_bytes)
            msg = f.read(msg_len)
            sd = SensorData()
            sd.ParseFromString(msg)
            if sd.HasField("image_data") and len(sd.image_data) and sd.image_data[0].HasField("image"):
                img_bytes = sd.image_data[0].image.raw_data
                img_np = np.frombuffer(img_bytes, dtype=np.uint8)
                frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
                if frame is not None:
                    yield frame

# achar uma pasta /Camera no subset
scenario_dir = next(DATA_ROOT.rglob("Camera"), None)
assert scenario_dir is not None, "Não encontrei /Camera no subset. Verifique se o ZIP foi extraído corretamente."

cam_file = next(Path(scenario_dir).glob("*.osi"))
print("🎥 Câmera .osi:", cam_file)

# gerar um preview curto (≤150 frames)
frames = []
for i, fr in enumerate(iter_camera_frames(cam_file)):
    frames.append(fr)
    if i >= 150: break

assert frames, "Não foi possível decodificar frames. Verifique a instalação do OSI e o arquivo .osi."

h, w = frames[0].shape[:2]
out_path = "/content/camera_preview.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw = cv2.VideoWriter(out_path, fourcc, 15, (w, h))
for fr in frames:
    vw.write(fr)
vw.release()
print("✅ Preview salvo em:", out_path)

In [ ]:
# CEL 7 — carregar intrínsecas do OpenLABEL e desenhar uma anotação no primeiro frame
import json, cv2, numpy as np
from pathlib import Path

openlabel_file = next(Path(scenario_dir).glob("*open*label*camera*.json"), None)
print("🧾 OpenLABEL:", openlabel_file)

with open(openlabel_file, "r") as f:
    ol = json.load(f)

# Matriz intrínseca K
K = np.array(
    ol["openlabel"]["frames"][0]["frame_properties"]["Streams"]["Camera1"]["intrinsics_pinhole"]["camera_matrix_3x4"]
).reshape(3,4)[:3,:3]

# Demonstração simples (texto). Para projeção 3D->2D (arestas), use as funções do cuboid_project.py
frame0 = frames[0].copy()
cv2.putText(frame0, "Cuboids: use funcoes do cuboid_project.py para projetar 3D->2D", (30,40),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
cv2.imwrite("/content/cuboids_demo.jpg", frame0)
print("✅ Exemplo salvo em /content/cuboids_demo.jpg")

In [ ]:
# CEL 8 — visualizar radar (cluster/object list) — precisa de arquivo em /Radar
from importlib import reload
import os
try:
    import radar_osi_plot as rplt
    reload(rplt)
    radar_dir = (Path(scenario_dir).parent / "Radar")
    if radar_dir.exists():
        radar_osi = next(radar_dir.glob("*.osi"))
        print("📡 Radar .osi:", radar_osi)
        # Abra o arquivo radar_osi_plot.py para ver como salvar vídeo (há exemplo lá).
    else:
        print("⚠️ Este caso do subset pode não ter Radar. Teste outro cenário ou use o dataset completo.")
except Exception as e:
    print("⚠️ Import do script de radar falhou:", e)

In [ ]:
# CEL 9 — (opcional) rodar YOLOv7 para DEMO
import os, cv2
os.makedirs("/content/frames", exist_ok=True)
for i, fr in enumerate(frames[:30]):
    cv2.imwrite(f"/content/frames/{i:04d}.jpg", fr)

!git clone -q https://github.com/WongKinYiu/yolov7.git
%cd /content/yolov7
!pip -q install -r requirements.txt
%cd /content/yolov7
!python detect.py --weights yolov7.pt --conf 0.25 --source /content/frames --save-txt --project /content/y7out --name det --exist-ok
print("✅ Saídas em /content/y7out/det")